In [1]:
from sklearn.ensemble import GradientBoostingRegressor
# 导入均方误差计算
from sklearn.metrics import mean_squared_error
# 导入标准化模块包
from sklearn.preprocessing import RobustScaler
# 导入划分数据集包，交叉验证包
from sklearn.model_selection import train_test_split,KFold,cross_val_score
# 导入Xgboost算法包
import xgboost as xgb
from  sklearn import datasets 
import pandas as pd 
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing  
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder, StandardScaler  

In [2]:
dataset_df = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')
# dataset_df.pop('Id')
# test_df.pop('Id')


# def handle_missing_values(df):
#     num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
#     cat_cols = [cname for cname in df.columns if df[cname].dtype == "object"]
#     values = {}
#     for a in cat_cols:
#         values[a] = 'UNKNOWN'
#     for a in num_cols:
#         values[a] = df[a].median()
#     df.fillna(value=values, inplace=True)
    
    
# handle_missing_values(dataset_df)
# handle_missing_values(test_df)

# print(dataset_df)

print(dataset_df['SalePrice'])
print('----------------')

y = dataset_df.pop('SalePrice')
# print(dataset_df.drop('SalePrice'))
# print(test_df)
all_data = pd.concat([dataset_df, test_df],axis=0)
# print(all_data)
# all_data = pd.get_dummies(all_data)
# print(all_data.columns)

# num_cols = [cname for cname in dataset_df.columns if dataset_df[cname].dtype in ['int64', 'float64']]
# cat_cols = [cname for cname in dataset_df.columns if dataset_df[cname].dtype == "object"]
    


0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
----------------


In [3]:
print(len(all_data))
print(dataset_df.columns.size)
print(test_df.columns.size)

2919
80
80


In [4]:
def handle_missing_values(df):
    num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
    cat_cols = [cname for cname in df.columns if df[cname].dtype == "object"]
    values = {}
    for a in cat_cols:
        values[a] = 'UNKNOWN'
    for a in num_cols:
        values[a] = df[a].median()
    df.fillna(value=values, inplace=True)
    

handle_missing_values(all_data)

all_data = pd.get_dummies(all_data)

In [5]:
all_data
len(all_data)

2919

In [20]:
train = all_data[all_data['Id'] < 1461]
train.pop('Id')

test = all_data[all_data['Id'] >= 1461]
Id = test.pop('Id')

In [18]:
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]}
other_params = {'learning_rate': 0.07, 'n_estimators': 800, 'max_depth': 4, 'min_child_weight': 3, 'seed': 0,
                    'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.1, 'reg_alpha': 3, 'reg_lambda': 1}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)

optimized_GBM.fit(train, y)
evalute_result = optimized_GBM.cv_results_['mean_test_score']
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))


print(optimized_GBM.best_estimator_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
每轮迭代运行结果:[0.8852689  0.8965886  0.89923674 0.88973721 0.87500432]
参数的最佳取值：{'learning_rate': 0.07}
最佳模型得分:0.8992367382840651
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.07, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             n_estimators=800, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [23]:
pred = optimized_GBM.best_estimator_.predict(test)

out = {
    'Id':Id,
    'SalePrice':pred
}

out = pd.DataFrame(out)

print(out)

out.to_csv('submisson1.csv',index=False)

        Id      SalePrice
0     1461  128944.609375
1     1462  165491.031250
2     1463  186386.656250
3     1464  197812.125000
4     1465  181064.828125
...    ...            ...
1454  2915   83923.789062
1455  2916   76248.320312
1456  2917  177560.671875
1457  2918  112513.664062
1458  2919  220990.953125

[1459 rows x 2 columns]
